# Notebook 2:  Data Cleaning 

In this notebook we merge our raw data tables from Notebook 1 and clean the player salaries.
1.  **Merging**: We update our 2020 free agent data with the 2021 next-season salaries, and also merge current-season (2021-22) stats and salaries together.  This gives us two new dataframes:  df2020_merged and df2021_merged.
2.  **Cleaning salaries**: We convert salaries into numerical form.  We also normalize salaries from all years to the 2021 salary cap for consistency.

## Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup, Comment
import requests
import lxml
import unicodedata
import json, pickle

## Utility Functions

In [2]:
def strip_accents_and_punctuation(text):
    '''Normalize player name spellings'''
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text).replace('.','').replace(',','').replace("'",'')

# (1) Merge Scraped Data

In [3]:
df2020 = pd.read_csv('./data/df2020_raw.csv')
df2021salaries = pd.read_csv('./data/dfcurrentsalaries.csv')
df2021stats = pd.read_csv('./data/dfcurrentstats.csv')
df2021urls = pd.read_csv('./data/dfplayer_to_url.csv')

## a. Update df2020 with 2021 salaries

In [4]:
df2020_merged = pd.merge(df2020, df2021salaries, left_on="Name", right_on="Name", how="left")
df2020_merged = df2020_merged.drop(columns=['NextSal', 'Unnamed: 0', 'CurrentTeam'])
df2020_merged = df2020_merged.rename(columns = {'CurrentSalary':'NextSal'})
df2020_merged = df2020_merged.set_index('NameYear')
df2020_merged.head(3)

,Name,Pos,Type,OTm,PrevStats,NTm,ID,Weight,Height,Age,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PrevSal,PrevYear,NextSal
NameYear,,,,,,,,,,,,,,,,,,,,,
Chris_Paul_2020,Chris Paul,G,UFA,PHO,"16.4 Pts, 4.5 Reb, 8.9 Ast",PHO,/players/p/paulch01.html,175.0,6-0,35.0,...,2.7,9.2,0.201,3.5,1.1,4.6,3.6,"$41,358,814",2020,"$30,800,000"
Kawhi_Leonard_2020,Kawhi Leonard,F,UFA,LAC,"24.8 Pts, 6.5 Reb, 5.2 Ast",LAC,/players/l/leonaka01.html,225.0,6-7,29.0,...,2.4,8.8,0.238,6.0,1.3,7.3,4.2,"$34,379,100",2020,"$39,344,900"
Enes_Freedom_2020,Enes Freedom,C,UFA,POR,"11.2 Pts, 11.0 Reb, 1.2 Ast",BOS,/players/k/kanteen01.html,250.0,6-10,28.0,...,1.7,7.5,0.204,2.1,-1.4,0.6,1.2,"$5,005,350",2020,"$2,641,691"


## b. Update df2021 with 2021 salaries and player urls

In [5]:
df2021_merged = pd.merge(df2021stats, df2021salaries, left_on="Player", right_on="Name", how="left")
df2021_merged = pd.merge(df2021_merged, df2021urls, left_on="Player", right_on="Name", how="left")

df2021_merged = df2021_merged.drop(columns=['Name_x', 'Name_y','Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','Rk'])
df2021_merged = df2021_merged.rename(columns = {'Player':'Name'})

df2021_merged.head(3)

,Name,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,CurrentSalary,CurrentTeam,ID
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,.439,...,11.3,14.9,.503,18.5,-0.2,2.5,.070,"$2,711,280",TOR,/players/a/achiupr01.html
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,.547,...,19.6,19.9,.560,12.0,2.0,6.8,.163,"$17,073,171",MEM,/players/a/adamsst01.html
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,.557,...,14.4,17.5,.608,25.0,2.7,7.2,.188,"$28,103,550",MIA,/players/a/adebaba01.html


## c. Update df2021 with current team and total stats

This is a little bit tricky because some players played for multiple teams.  As an example, Nickeil Alexander-Walker switched teams mid-season and thus has three data rows:

* Name    /   Team
* Nickeil Alexander-Walker  /    TOT
* Nickeil Alexander-Walker  /    NOP
* Nickeil Alexander-Walker  /    UTA

We want to keep his **season stats** from the first row (TOT) but also indicate his **current team** from the last row (UTA).  The same pattern holds for all players who played for multiple teams.

In [6]:
# Create current team name filter 
justteam = ~df2021_merged.duplicated(subset=['Name'],keep='last')
# Create total stats filter
juststats = ~df2021_merged.duplicated(subset=['Name'],keep='first')


df2021_merged = (pd.merge(df2021_merged[juststats], df2021_merged[justteam][['Name','Tm']], 
                          left_on="Name", right_on="Name", how="left")
                            .rename(columns ={'Tm_x':'TeamOrTot', 'Tm_y':'Team'}) 
         )

df2021_merged['NameYear'] = df2021_merged['Name'].apply(strip_accents_and_punctuation).str.replace(' ','_') + '_2021' 
df2021_merged = df2021_merged.set_index('NameYear')

Minor bookkeeping:
Update a few Teams with correct info (only applies to 15 players, not that important):


In [7]:
mask = df2021_merged.CurrentTeam.notna() & (df2021_merged.CurrentTeam != df2021_merged.Team)
df2021_merged.loc[mask,'Team'] = df2021_merged.loc[mask,'CurrentTeam']

df2021_merged = df2021_merged.drop(columns=["CurrentTeam"])
df2021_merged = df2021_merged.rename(columns= {'CurrentSalary': 'Salary'})

df2021_merged.head(5)

,Name,Pos,Age,TeamOrTot,G,GS,MP,FG,FGA,FG%,...,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,Salary,ID,Team
NameYear,,,,,,,,,,,,,,,,,,,,,
Precious_Achiuwa_2021,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,.439,...,11.3,14.9,.503,18.5,-0.2,2.5,.070,"$2,711,280",/players/a/achiupr01.html,TOR
Steven_Adams_2021,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,.547,...,19.6,19.9,.560,12.0,2.0,6.8,.163,"$17,073,171",/players/a/adamsst01.html,MEM
Bam_Adebayo_2021,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,.557,...,14.4,17.5,.608,25.0,2.7,7.2,.188,"$28,103,550",/players/a/adebaba01.html,MIA
Santi_Aldama_2021,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,.402,...,9.9,12.6,.452,18.4,-0.3,0.3,.044,"$1,994,520",/players/a/aldamsa01.html,MEM
LaMarcus_Aldridge_2021,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,.550,...,8.0,13.4,.604,22.4,0.7,3.1,.141,"$2,641,691",/players/a/aldrila01.html,BRK


## Save merged data

In [8]:
df2020_merged.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/df2020_merged.csv')
df2021_merged.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/df2021_merged.csv')

# (2) Clean salary data

## Load files

In [9]:
df2016_raw = pd.read_csv('./data/df2016_raw.csv')
df2017_raw = pd.read_csv('./data/df2017_raw.csv')
df2018_raw = pd.read_csv('./data/df2018_raw.csv')
df2019_raw = pd.read_csv('./data/df2019_raw.csv')
df2020_merged = pd.read_csv('./data/df2020_merged.csv')

df  = pd.concat([df2016_raw, df2017_raw, df2018_raw, df2019_raw, df2020_merged])
df = df.set_index('NameYear')

df2021 = pd.read_csv('./data/df2021_merged.csv')
df2021 = df2021.set_index('NameYear')
df2021 = df2021.drop("Player_2021")  #spurious entry

## Delete unnecessary entries

We first employ a very light common-sense filter on our entries.  To train our player value model, we wish to keep only players who: 
1.  Played in at least one game the previous year
2.  Had a salary the next year (i.e. actually received a contract in free agency)

In [10]:
#  1. Mask for players who played in at least one game the previous year

df['G'] = df['G'].apply(lambda s: pd.to_numeric(s, errors='coerce'))
mask1 = df['G'].isna()
df = df[~mask1]

#  2. Mask for players who had a salary the next year

mask2  = (df['NextSal'] == '0') | df['NextSal'].isna()  
df = df[~mask2]
df.sample(3)

,Name,Pos,Type,OTm,PrevStats,NTm,ID,Weight,Height,Age,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,PrevSal,NextSal,PrevYear
NameYear,,,,,,,,,,,,,,,,,,,,,
Juan_Toscano-Anderson_2019,Juan Toscano-Anderson,F,UFA,GSW,"5.3 Pts, 4.0 Reb, 2.0 Ast",NaN,/players/t/toscaju01.html,209.0,6-6,26.0,...,0.2,0.2,0.030,-4.1,0.5,-3.6,-0.1,"$350,189","$358,295< $Minimum",2019
Joe_Chealey_2018,Joe Chealey,G,UFA,CHO,"2.0 Pts, 0.0 Reb, 1.0 Ast",CHO,/players/c/chealjo01.html,190.0,6-3,23.0,...,0.0,0.0,-0.080,-8.5,-6.0,-14.6,0.0,< $Minimum,"$163,356",2018
Luc_Mbah_a_Moute_2018,Luc Mbah a Moute,F,UFA,NaN,"5.0 Pts, 1.8 Reb, 0.5 Ast",NaN,/players/m/mbahalu01.html,230.0,6-8,32.0,...,0.0,0.0,0.009,-2.9,0.0,-3.0,0.0,"$4,320,500","$289,803",2018


### Clean salries and normalize by salary cap

Since we are looking at data for multiple years, it is important to normalize by salary cap for apples-to-apples comparisons in our model.  We chose the 2021 salary cap for simplicity.

In [11]:
def clean_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
    '''
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('(TW)','').replace('Minimum','0')
            elem =  float(elem)
            newlist.append(elem)
        except:
            continue
            
    salary = sum(newlist)/1000000
    
    return salary

# Salary Cap Data taken from spotrac.com/nba/cba
salarycap = {2016:   94.143,  2017: 99.093,   2018: 101.869,   2019:  109.14,   
             2020:   109.14,  2021: 112.414}   

def clean_and_norm_salary(sal, year):
    ''' Adds all salaries earned per year into one salary (in millions of dollars)
        "< $Minimum" is treated as $0
        Then normalizes by salary cap (in 2021 salary cap dollars)
    '''
    newlist = [0]
    for elem in sal.split('$'):
        try:
            elem = elem.replace('<','').replace(',','').replace('Minimum','0')
            elem =  float(elem)
            newlist.append(elem)
        except:
            continue
            
    salary = sum(newlist)/1000000
    salary = salary*salarycap[2021]/salarycap[year]
        
    return salary

for index, row in df.iterrows():
    
    prevyear = df.loc[index, 'PrevYear']
    nextyear = df.loc[index, 'PrevYear'] + 1 

    prevsal = df.loc[index, 'PrevSal']
    nextsal = df.loc[index, 'NextSal']
    
    df.loc[index, 'PrevSalClean'] = clean_salary(prevsal, prevyear)
    df.loc[index, 'NextSalClean'] = clean_salary(nextsal, nextyear)
    df.loc[index, 'PrevSalNorm'] = clean_and_norm_salary(prevsal, prevyear)
    df.loc[index, 'NextSalNorm'] = clean_and_norm_salary(nextsal, nextyear)

cols = ['NextSal', 'NextSalClean','NextSalNorm']
df[cols].sample(8)

,NextSal,NextSalClean,NextSalNorm
NameYear,,,
Terence_Davis_2020,"$4,000,000",4.000000,4.000000
Paul_Pierce_2016,"$1,096,080",1.096080,1.243425
Vince_Carter_2016,"$8,000,000",8.000000,9.075434
Derrick_Jones_Jr_2017,"$1,512,601",1.512601,1.669178
Kyrie_Irving_2018,"$31,742,000",31.742000,32.694202
Mike_Muscala_2018,"$2,028,594",2.028594,2.089448
Wenyen_Gabriel_2020,"$29,814",0.029814,0.029814
Nemanja_Bjelica_2020,"$2,089,448",2.089448,2.089448


In [12]:
for index, row in df2021.iterrows():
    
    salary = df2021.loc[index, 'Salary']
    try:
        df2021.loc[index, 'SalClean'] = clean_salary(salary, 2021)
    except:
        df2021.loc[index, 'SalClean'] = np.nan
        
    try:
        df2021.loc[index, 'SalNorm'] = norm_salary(salary, 2021)
    except:
        df2021.loc[index, 'SalNorm'] = np.nan

df2021.sample(3)

,Name,Pos,Age,TeamOrTot,G,GS,MP,FG,FGA,FG%,...,TS%,USG%,VORP,WS,WS/48,Salary,ID,Team,SalClean,SalNorm
NameYear,,,,,,,,,,,,,,,,,,,,,
Eric_Paschall_2021,Eric Paschall,PF,25,UTA,58,3,12.7,2.0,4.1,.485,...,.612,17.7,0.3,2.0,.128,"$1,782,621",/players/p/pascher01.html,UTA,1.782621,NaN
Paul_Watson_2021,Paul Watson,SF,27,OKC,9,3,17.3,1.3,3.9,.343,...,.432,11.0,-0.1,0.0,.001,NaN,/players/w/watsopa01.html,OKC,NaN,NaN
BJ_Johnson_2021,B.J. Johnson,SF,26,ORL,4,0,16.3,2.8,6.3,.440,...,.520,16.7,0.0,0.1,.065,NaN,/players/j/johnsbj01.html,ORL,NaN,NaN


## Save cleaned data

In [13]:
df.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/cleaned_pastyears.csv')
df2021.to_csv(r'/Users/andrei/Dropbox/Metis/HoopsHero/data/cleaned_currentyear.csv')